<a href="https://colab.research.google.com/github/Tomo-for-lab/Code-of-automating-DE/blob/main/analysis/arm_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai weave

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import re
import ipykernel
from notebook.notebookapp import list_running_servers
import requests
from urllib.parse import urljoin
import json
import os
from google.colab import auth
from google.colab import userdata
from googleapiclient.discovery import build
import glob

from openai import OpenAI
from openai import AzureOpenAI
import pandas as pd
import numpy as np
from tqdm import tqdm

Change the working directory to the path of the current notebook

In [ ]:
from google.colab import drive
# Mount Google Drive to access files stored in it
drive.mount('/content/drive')


def get_notebook_path():
    # Get the path of the current Jupyter notebook
    kernel_id = re.search('kernel-(.*).json', ipykernel.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'), params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

def get_folder_path(folder_id):
    # Recursively get the full path of a folder given its ID
    if folder_id:
        folder = drive_service.files().get(fileId=folder_id, fields="name, parents").execute()
        folder_name = folder.get('name')
        parents = folder.get('parents')
        if parents:
            parent_path = get_folder_path(parents[0])
            return parent_path + '/' + folder_name
        else:
            return folder_name
    return ''

def get_file_path(file_id):
    # Recursively get the full path of a file given its ID
    file = drive_service.files().get(fileId=file_id, fields="name, parents").execute()
    file_name = file.get('name')
    parents = file.get('parents')
    if parents:
        parent_id = parents[0]
        parent_path = get_folder_path(parent_id)
        return parent_path
    else:
        return file_name

# Get the path of the current notebook
notebook_path = get_notebook_path()


# Authenticate and initialize the Google Drive API
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Extract the file ID from the notebook path (assumes file ID is part of the path)
file_id = re.search(r'fileId=(\w+)', notebook_path).group(1)

# Get the full path of the file using its ID
file_path = get_file_path(file_id)
if 'マイドライブ' in file_path:
    converted_path = re.sub(r'(^|/)マイドライブ($|/)', '/content/drive/MyDrive/', file_path)
elif 'MyDrive' in file_path:
    converted_path = re.sub(r'(^|/)MyDrive($|/)', '/content/drive/MyDrive/', file_path)
else:
    converted_path = '/content/drive/MyDrive/' + file_path

# Change the working directory to the converted path
os.chdir(converted_path)

Mounted at /content/drive


In [ ]:
import weave

# Preparation of Azure for GPT

In [ ]:
#deploy names
deployment_ids = ["gpt-4o","gpt-4-turbo","gpt-3.5-turbo","text-embedding-ada-002"]

#If you use Azure
#key
API_KEY = userdata.get('Azure_API_key')
RESOURCE_ENDPOINT = userdata.get('RESOURCE_ENDPOINT')
AZURE_API_version = userdata.get('AZURE_API_version')

#client
client = AzureOpenAI(
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version=AZURE_API_version,
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=RESOURCE_ENDPOINT,
    api_key=API_KEY
)


#Create a dict of human-extracted arm

In [ ]:
# Specify the file path and sheet name for the Excel file
file_path = 'DE.xlsx'
sheet_name = 'R1_FINAL'

# Read the Excel file, specifying header=1 because the first row is blank
df = pd.read_excel(file_path, sheet_name=sheet_name, header=1)

# Initialize a dictionary to store the results
study_dict = {}

for index, row in df.iterrows():
    study_name = row['study'] # Get the study name from the 'study' column
    arm_name = row['Arm'] # Get the arm name from the 'Arm' column
    if pd.notna(study_name):
        if study_name not in study_dict:
            study_dict[study_name] = []
        # Append the arm name to the list corresponding to the study name
        study_dict[study_name].append(arm_name)

print(len(study_dict))

295


# Define required function

## Function to create tools for matching names of arms extracted by GPT and human

In [ ]:
def make_tools(GPT_arms_list):
  tools_properties = {}

  # Iterate over the list of arm names and add them to the tools_properties dictionary
  for arm_name in GPT_arms_list:
    tools_properties[arm_name] = {'type': 'string', 'description': 'Answer the matched word.'} # Provide a description for the property

  tools = [{'type': 'function',
      'function':
          {'name': 'word_matching',
          'description': 'Answer the word matched to the name',
          'parameters': {
              'type': 'object',
            'properties': tools_properties
          }
      }
  }
  ]

  tools[0]['function']['parameters']['required'] = GPT_arms_list
  return tools

## Function for GPT function calling

In [ ]:
@weave.op() # 🐝
def GPT_function_calling(tools, messages, tools_name):
  response = client.chat.completions.create(
        model=deployment_ids[0],
        messages=messages,
        tools=tools,
        tool_choice =  {"type": "function", "function": {"name": tools_name}},
        temperature=0
    )
  return response

#Match arm name

In [ ]:
# Create initial DataFrame
columns = ["n_index", "fold_index", "eval_index", "paper_name", "GPT_1", "GPT_2", "GPT_3", "GPT_4", "GPT_5",
           "human_1", "human_2", "human_3", "human_4", "human_5", "key_1", "key_1_matched", "value_1", "value_1_matched",
           "key_2", "key_2_matched", "value_2", "value_2_matched", "key_3", "key_3_matched", "value_3", "value_3_matched",
           "key_4", "key_4_matched", "value_4", "value_4_matched", "key_5", "key_5_matched", "value_5", "value_5_matched", "error"]
df_output = pd.DataFrame(columns=columns)
df_output.loc[0] = {col: "" for col in columns}

In [ ]:
weave.init("arm_matching_GPT4o")

In [ ]:
#Specify the path where the extracted data were stored
extracted_data_dir = "extracted_data_log_GPT_v8"

#Specify developed description obtained by which method is used
shot_name = "chat"
for n_index in range(5,6):
  for fold_index in range(10):
    for eval_index in range(5):
      print("n_index: ", n_index)
      print("fold_index: ", fold_index)
      print("eval_index: ", eval_index)

      # Create a new row with specified columns
      new_row = pd.DataFrame(columns=columns)
      new_row.loc[0] = {col: "" for col in columns}
      new_row.loc[0, "n_index"] = n_index
      new_row.loc[0, "fold_index"] = fold_index
      new_row.loc[0, "eval_index"] = eval_index

      try:
        # Construct the path to the extracted data JSON file
        extracted_data_path = os.path.join(extracted_data_dir, f"{shot_name}/{n_index}_paper/{fold_index}_fold/extracted_data_{eval_index}.json")
        with open(extracted_data_path, 'r', encoding='utf-8') as file:
          # Load the JSON file
          extracted_data = json.load(file)

      except Exception as e:
        file_name = "extracted_data_loading"
        new_row.loc[0, "error"] = f"{file_name}: {str(e)}"
        df_output = pd.concat([df_output, new_row], ignore_index=True)
        print("error_type: ", file_name)
        continue;

      # Extract the paper name from the keys of the extracted data
      paper_name = list(extracted_data.keys())[0]
      new_row.loc[0, "paper_name"] = paper_name

      # Get the list of arm names from the extracted data
      GPT_arms_list = list(extracted_data[paper_name].keys())

      try:
        # Retrieve the human-provided list of arm names for the paper
        human_arms_list = study_dict[paper_name]
      except Exception as e:
        file_name = "no_paper_name"
        new_row.loc[0, "error"] = f"{file_name}: {str(e)}"
        df_output = pd.concat([df_output, new_row], ignore_index=True)
        print("error_type: ", file_name)
        continue;

      # Store the arm names extracted by GPT and human in the new row
      for i in range(len(GPT_arms_list)):
          new_row.loc[0, f"GPT_{i+1}"] = GPT_arms_list[i]
      for i in range(len(human_arms_list)):
          new_row.loc[0, f"human_{i+1}"] = human_arms_list[i]

      # Check if the number of arms match between GPT and human data
      if len(GPT_arms_list) != len(human_arms_list):
        file_name = "number_of_arms"
        e = "The numbers of the arms in both lists are not matched.\n\n"
        new_row.loc[0, "error"] = f"{file_name}: {str(e)}"
        df_output = pd.concat([df_output, new_row], ignore_index=True)
        print("error_type: ", file_name)
        continue;

      # Create tools for word matching
      try:
        tools = make_tools(GPT_arms_list)
      except Exception as e:
        file_name = "making_tools"
        new_row.loc[0, "error"] = f"{file_name}: {str(e)}"
        df_output = pd.concat([df_output, new_row], ignore_index=True)
        print("error_type: ", file_name)
        continue;

      # Prepare messages for GPT function calling
      messages=[]
      system_content = "Use the word_matching tool in your response."
      user_content = """You have to answer the most matched word for each name in the word_matching tool.
      Use the word_matching tool in your response.
      Below is a word list from which you can choose and answer.\n
      """

      for arms_list_num in range(len(human_arms_list)):
        user_content += human_arms_list[arms_list_num]
        user_content += "\n"

      messages.append({"role": "system", "content": system_content})
      messages.append({"role": "user", "content": user_content})

      # Call GPT function for word matching
      try:
        response = GPT_function_calling(tools, messages, "word_matching")

      except Exception as e:
        file_name = "GPT_chat"
        new_row.loc[0, "error"] = f"{file_name}: {str(e)}"
        df_output = pd.concat([df_output, new_row], ignore_index=True)
        print("error_type: ", file_name)
        continue;

      # Process the response from GPT function calling
      try:
        tools_data= json.loads(response.choices[0].message.tool_calls[0].function.arguments)
        # Store the tools data in the new row
        i = 0
        for key, value in tools_data.items():
            if i < 5:  # Store up to 5 key-value pairs
                new_row.loc[0, f"key_{i+1}"] = key
                new_row.loc[0, f"value_{i+1}"] = value
                new_row.loc[0, f"key_{i+1}_matched"] = ""
                new_row.loc[0, f"value_{i+1}_matched"] = ""
            i += 1
        df_output = pd.concat([df_output, new_row], ignore_index=True)

      except Exception as e:
        file_name = "output_loading"
        new_row.loc[0, "error"] = f"{file_name}: {str(e)}"
        df_output = pd.concat([df_output, new_row], ignore_index=True)
        print("error_type: ", file_name)
        continue;

# Output the DataFrame to an Excel file
excel_path_updated = "arm_matched_list.xlsx"
df_output.to_excel(excel_path_updated, index=False)

n_index:  5
fold_index:  0
eval_index:  0
🍩 https://wandb.ai/srws-psg/arm_matching_GPT4o/r/call/5420b846-a6f0-4ff0-aa0d-06822805e065
n_index:  5
fold_index:  0
eval_index:  1
🍩 https://wandb.ai/srws-psg/arm_matching_GPT4o/r/call/097e1e1a-2f8d-4e87-91b9-d7aa9456cbba
n_index:  5
fold_index:  0
eval_index:  2
🍩 https://wandb.ai/srws-psg/arm_matching_GPT4o/r/call/a742951d-1fb3-4649-a4b8-477515a2b507
n_index:  5
fold_index:  0
eval_index:  3
🍩 https://wandb.ai/srws-psg/arm_matching_GPT4o/r/call/91ac790f-873a-4137-882b-40ed558b1366
n_index:  5
fold_index:  0
eval_index:  4
🍩 https://wandb.ai/srws-psg/arm_matching_GPT4o/r/call/c6f52fcb-aaef-45ed-ab73-92cb475d8602
n_index:  5
fold_index:  1
eval_index:  0
🍩 https://wandb.ai/srws-psg/arm_matching_GPT4o/r/call/87ae2aeb-a79d-45e2-9e26-6c7796078558
n_index:  5
fold_index:  1
eval_index:  1
🍩 https://wandb.ai/srws-psg/arm_matching_GPT4o/r/call/f3d70020-e563-490b-b503-f5814c0ef0fe
n_index:  5
fold_index:  1
eval_index:  2
🍩 https://wandb.ai/srws-psg